In [1]:
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
from scipy import signal, integrate
import datetime
import configparser

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib
from matplotlib.colors import BoundaryNorm, LogNorm
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from matplotlib.ticker import MaxNLocator

# - for reloading libraries/modules - #
import importlib
import multiprocessing

import subroutines
import vis_eulag
#importlib.reload(subroutines)
#importlib.reload(vis_eulag)
#from vis_eulag import *

import cmaps

if os.path.exists('latex_default.mplstyle'):
    plt.style.use('latex_default.mplstyle')

In [2]:
def preprocess_eulag_output(config):
    """Process EULAG output"""
    folder = "/scratch/b/b309199/"
    fileLocation   = folder + "translb_lid_pnj"
    fileLocation   = folder + "translb_lid_cwind"
    fileLocation   = folder + "statlb_lid_cwind"
    fileLocation   = folder + "wavebreak_cwind"
    fileLocation   = folder + "wavebreak_cwind2"
    fileLocation   = folder + "wavebreak_cwind_hr"
    #fileLocation   = folder + "wavebreak_pnj"
    #fileLocation   = folder + "testtopo"
    fileLocation   = folder + "twomtns"
    env_fileName   = "env.nc"
    tapes_fileName = "tapes.nc"
    grid_fileName  = "grd.nc"
    xz_fileName    = "xzslc.nc"
    xy_fileName    = "xyslc.nc"
    env_path = os.path.join(fileLocation, env_fileName)
    tapes_path = os.path.join(fileLocation, tapes_fileName)
    grid_path = os.path.join(fileLocation, grid_fileName)
    xz_path = os.path.join(fileLocation, xz_fileName)
    xy_path = os.path.join(fileLocation, xy_fileName)

    ds_full = xr.open_dataset(tapes_path)
    ds_env = xr.open_dataset(env_path)
    ds_grid = xr.open_dataset(grid_path)
    ds      = xr.open_dataset(xz_path)
    dsxy    = xr.open_dataset(xy_path)
    ds      = ds.assign_coords({'xcr':ds_grid['xcr']/1000, 'ycr':ds_grid['ycr']/1000, 'zcr':ds['zcr']/1000})
    dsxy['zcr'] = dsxy['zcr'] / 1000
    # ---- Sim parameters -------------- # 
    ds.attrs = ds_grid.attrs
    ds.attrs['bv'] = ds.attrs['bv'].round(3)
    ds.attrs['nx'] = np.shape(ds_full['w'])[3]
    ds.attrs['ny'] = np.shape(ds_full['w'])[2]
    ds.attrs['nz'] = np.shape(ds_full['w'])[1]

    """Lidar outputs with high temporal resolution"""
    lid_files_list = glob.glob(os.path.join(fileLocation, "lid_*"))
    ## print("[i]   Lidar list: ", lid_files_list)
    ds_lid_list = []
    for lid_file in lid_files_list:
        ds_lid = xr.open_dataset(lid_file)
        ds_lid['time'] = ds_lid.t * ds.nlid * ds.dt00/3600
        ds_lid['time'] = ds_lid['time'].expand_dims({'z':ds_lid.z}, axis=1)
        ds_lid['zcr'] = ds_lid['zcr']/1000
        ## ds_lid = vis_eulag.process_ds_lid(ds_lid, ds)
        ds_lid['location'] = lid_file.split("/")[-1][:-3]
        ds_lid_list.append(ds_lid)
        
    return ds, dsxy, ds_env, ds_lid_list, ds_full

config_file  = "settings_translb3D.ini"
config = configparser.ConfigParser()
config.read(config_file)
ds, dsxy, ds_env, ds_lid_list, ds_full = preprocess_eulag_output(config)
print("[i]  Done")

[i]  Done


In [4]:
#%%capture
""""Animation of xz-slice with virtual lidar observation"""
def vis_xzslice(config,t):
    """EULAG output"""
    ds, dsxy, ds_env, ds_lid_list, ds_full = preprocess_eulag_output(config)
    
    """Visualize theta prime or other vars in xzslice with virtual lidar"""
    gskw = {'hspace':0.14, 'wspace':0.06, 'height_ratios': [7,7,0.1,2]} #  , 'width_ratios': [5,5]}
    fig, axes = plt.subplots(4,2,figsize=(9,10), gridspec_kw=gskw)
    axes[-2,0].set_axis_off()
    axes[-2,1].set_axis_off()
    axes[-1,0].set_axis_off()
    axes[-1,1].set_axis_off()
    axes[1,1].set_axis_off()
    
    """Parameter/Settings"""
    c1 = 'black'
    c2 = 'forestgreen'
    c3 = 'lightgrey'
    c4 = 'darkorchid'
    lw_1 = 2
    lw_2 = 1.5
    lw_sponge = 1.5
    xp = 0.04
    yp = 0.89
    alpha_box = 0.9
    alpha_sponge = 0.7
    numb_str = ['a','b','c','d','e','f']
    surf_factor = 10
    show_lid = True
    
    if config.getboolean("GENERAL","STRATOS"):
        thlev=np.exp(5+0.03*np.arange(1,250,10)) # for N=0.02
    else:
        thlev=np.exp(5+0.03*np.arange(1,100)) # 90 is needed for atmosphere up to 100km
        # thlev=np.exp(5+0.03*np.arange(1,100,0.5)) # more isentropes for troposphere visualization
    
    """Limits"""
    if show_lid:
        ds_lid = ds_lid_list[0]
        XLIM_LID = [0,ds_lid.time.max().values]
    
    tref = 28
    # tstamp_ref = tref * (ds_lid.time.max().values / (np.shape(ds['th'])[0]-1))
    # tstamp     = t * (ds_lid.time.max().values / (np.shape(ds['th'])[0]-1))
    tstamp_ref = tref * ds.nslice * ds.dt00 / 3600
    tstamp     = t    * ds.nslice * ds.dt00 / 3600

    xloc = int(str(ds_lid['location'].values)[4:9]) * ds.dx00/1000
    if ds.ibcx == 0:
        xloc = xloc - ds.nx/2 * ds.dx00/1000
    # yloc = (int(str(ds_lid['location'].values)[-3:]) - int(ds.ny/2)) * ds.dy00/1000
    y  = int(ds.ny/2)
    zl = -4
    zu = ds.zcr.max().values
    zu = 60
    zsponge = [ds.zab/1000, zu]
    ZLIM    = [zl,zu]
    XLIM    = [ds.xcr.min().values,ds.xcr.max().values]
    ## print("[i]   Time: {}h, Lidar location: {}km, Sponge: {}".format(tstamp, xloc, zsponge))
    
    """Colormap / CLEVELS"""
    cmap   = cmaps.get_wave_cmap()
    clev   = [-32,-16,-8,-4,-2,-1,-0.5,0.5,1,2,4,8,16,32]
    clev_l = [-16,-4,-1,1,4,16]
    clev, clev_l = subroutines.get_colormap_bins_and_labels(max_level=0.8)
    norm = BoundaryNorm(boundaries=clev , ncolors=cmap.N, clip=True)

    ax0 = axes[0,0] # xz
    ax1 = axes[0,1] # lid
    ax2 = axes[1,0] # xy
    ax0.grid(False)
    ax1.grid(False)
    
    """Plot xz-slice"""
    #var = ds.th[t,:,:]-ds.th[tref,:,:]
    var = ds.th[t,:,:]
    pcMesh0 = ax0.contourf(ds.xcr.expand_dims({'z':ds.z},axis=0)[:,y,:], ds.zcr[t,:,:], var,
                            cmap=cmap, norm=norm, levels=clev, extend='both')

    isentropes = ax0.contour(ds.xcr.expand_dims({'z':ds.z},axis=0)[:,y,:], ds.zcr[t,:,:], ds['the'][t,:,:]+ds['th'][t,:,:], 
                             colors='k', alpha=0.7, levels=thlev)
    ax0.plot(ds.xcr[y], surf_factor*ds.zcr[t,0,:], lw=2, color='black')
    ax0.yaxis.set_major_locator(MultipleLocator(10))
    ax0.xaxis.set_minor_locator(AutoMinorLocator())
    ax0.yaxis.set_minor_locator(AutoMinorLocator())
    ax0.tick_params(labelbottom=False,labeltop=False)
    ax0.set_xlabel('streamwise x / km') # change to longitudes, latitude 10$^3$
    ax0.xaxis.set_label_position('top') 
    
    ##ax0.set_ylabel('altitude z-z$_{trp}$ / km')
    ax0.set_ylabel('altitude z / km')
    ax0.set_xlim(XLIM)
    ax0.set_ylim(ZLIM)
    ax0.vlines(x=[xloc], ymin=zl,ymax=zu, colors=c1, lw=lw_1, ls='--')
    #ax0.axhline(y=dsxy.zcr[0,0,0].values, color=c1, lw=lw_1, ls='--')
    ax0.axhline(y=dsxy.zcr[0,0,0].values, color=c1, lw=lw_1, ls='--')
    
    """Wind axis"""
    ax_wind = ax0.twiny()
    x=0
    ax_wind.plot(ds['ue'][t,:,x], ds.zcr[t,:,x], lw=2.5, ls='--', color=c4, label='$u_e$')
    ax_wind.plot(np.mean(ds['u'][t,:,:],axis=1), ds.zcr[t,:,x], lw=1.5, ls='-', color=c4, label='$u_e$')
    ax_wind.text(-0.05,-0.052,'u / m$\,$s$^{-1}$',color=c4,transform=ax_wind.transAxes,fontsize=9)
    ##ax_wind.set_xlabel('$\hat{u}$ / m$\,$s$^{-1}$')
    ax_wind.set_xlim([-15,400])
    ax_wind.vlines(x=[0], ymin=0,ymax=zu, colors=c4, lw=0.75, ls='-.')
    ##ax_wind.xaxis.set_major_locator(MultipleLocator(50))
    ##wind_xticks = ax_wind.get_xticks()
    plt.xticks([50,100], fontsize=8, fontweight='normal', visible=True)
    #plt.xticks(wind_xticks[1:3], labels=['50', '100ms$^{-1}$'], fontsize=9, fontweight='normal', visible=True)
    ax_wind.tick_params(axis='x', which='both', top=False, bottom=True, labelbottom=True,labeltop=False, colors=c4)
    ax_wind.xaxis.label.set_color(c4)
    # ax_wind.tick_params(axis='x', )
    # direction="out", pad=-20
    
    ax0.tick_params(which='both', top=True, right=True, bottom=False, labelbottom=False,labeltop=True)
    
    """Plot xy-slice"""
    var = dsxy.th[t,:,:]
    ax2.contourf(ds.xcr, ds.ycr, var,
                            cmap=cmap, norm=norm, levels=clev, extend='both')
    ### - Topography - ###
    if ds.amp < 0:
        topo_levels=np.linspace(surf_factor*ds.amp,-surf_factor*ds.amp,12)
    else: 
        topo_levels=np.linspace(-surf_factor*ds.amp,surf_factor*ds.amp,12)
    isentropes = ax2.contour(ds.xcr, ds.ycr, surf_factor*dsxy.zcrtopo[t,:,:], colors='k', levels=topo_levels, linewidths=1)
    ##isentropes = ax2.contour(ds.xcr, ds.ycr, -surf_factor*ds_env.zcr[-1,0,:,:], colors='k', levels=topo_levels, linewidths=1)
    
    # ax2.yaxis.set_major_locator(MultipleLocator(10))
    ax2.xaxis.set_minor_locator(AutoMinorLocator())
    ax2.yaxis.set_minor_locator(AutoMinorLocator())
    ax2.set_xlabel('streamwise x / km')
    ax2.set_ylabel('spanwise x / km')
    ax2.set_xlim(XLIM)
    #ax2.set_ylim(ZLIM)
    ax2.grid()
    ## include horizontal line in xz plot and vice versa in xy 
    
    """Plot virtual lidar"""
    if show_lid:
        ## tref_lid = np.where(ds_lid.time.values[:,0] == tstamp_ref)[0]
        ## var = ds_lid.th-ds_lid.th[tref_lid]
        var = ds_lid.th
        pcMesh1 = ax1.contourf(ds_lid.time, ds_lid.zcr, var, levels=clev,
                                cmap=cmap, norm=norm, extend='both')

        isentropes = ax1.contour(ds_lid.time, ds_lid.zcr, ds_lid.the + ds_lid.th, colors='k', levels=thlev)
        # ax.clabel(isentropes, thlev[1::], fontsize=8, fmt='%1.0f K', inline_spacing=1, inline=True, 
        #             manual=[(8,ds.zcr[t,10,0,x]), (8,ds.zcr[t,-15,0,x])]) # ha='left', thlev[1::3]

        ax1.plot(ds_lid.time, surf_factor*ds_lid.zcr[:,0], lw=2, color='black')
    ax1.yaxis.set_major_locator(MultipleLocator(10))
    ax1.xaxis.set_minor_locator(AutoMinorLocator())
    ax1.yaxis.set_minor_locator(AutoMinorLocator())
    ax1.tick_params(labelbottom=False,labeltop=True, labelleft=False)
    ax1.xaxis.set_label_position('top')
    ax1.set_xlabel('time / h') # change to longitudes, latitude
    ax1.set_xlim(XLIM_LID)
    ax1.set_ylim(ZLIM)
    ax1.vlines(x=[tstamp], ymin=zl,ymax=zu, colors=c1, lw=lw_1, ls='--')

    """Sponge layer"""
    #ax0.axhline(y=48, lw=lw_2,ls='--',color='grey')
    #ax1.axhline(y=48, lw=lw_2,ls='--',color='grey')
    if zsponge[0] > 0:
        ax0.fill_between(XLIM, [zsponge[1],zsponge[1]], [zsponge[0],zsponge[0]], facecolor=c3, alpha=alpha_sponge)
        ax1.fill_between(XLIM_LID, [zsponge[1],zsponge[1]], [zsponge[0],zsponge[0]], facecolor=c3, alpha=alpha_sponge)
        ##sponge_label = r'$\uparrow$ sponge layer $\uparrow$'
        ##axes[0,0].text(0.6, 0.72, sponge_label, transform=axes[0,0].transAxes, weight='bold', color='grey')

    # - Text - #
    #tstr = 'T: ' + str(12*t1) + 'h'
    #lidstr = 'Lidar @' + str(int(xloc)) + 'km'
    #tt1=ax_wind.text(xp, yp, tstr, transform=ax_wind.transAxes, weight='bold', color=c2,backgroundcolor='1')
    #tt2=ax1.text(xp, yp, lidstr, transform=ax1.transAxes, weight='bold', color=c1,backgroundcolor='1')
    #tt1.set_bbox(dict(facecolor='white', alpha=alpha_box,edgecolor=c2,lw=lw_2))
    #tt2.set_bbox(dict(facecolor='white', alpha=alpha_box,edgecolor=c1,lw=lw_2))

    # - Numbering - #
    #xpp = 0.93
    #ypp = 0.89
    #ax0.text(xpp, ypp, numb_str[2*i], transform=ax0.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    #ax1.text(xpp, ypp, numb_str[2*i+1], transform=ax1.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})


    """Colorbar"""
    cbar = fig.colorbar(pcMesh0, ax=axes[-1,:], location='bottom', shrink=0.67, fraction=1, ticks=clev_l, pad=0, extend='both') #  pad=0.15 default
    cbar.set_label(r"$\Theta'$ / K")

    """Save figure"""
    output_folder = "./data/xzslice"
    os.makedirs(output_folder,exist_ok=True)
    if t<10:
        buffer = "00"
    elif t<100:
        buffer = "0"
    else:
        buffer = ""
    fig_title = "xzslice_" + buffer + str(t) + ".png"
    fig.savefig(os.path.join(output_folder,fig_title), facecolor='w', edgecolor='w',
                    format='png', dpi=120, bbox_inches='tight')

#vis_xzslice(config,ds,ds_lid_list,28)
config['GENERAL']['NTASKS'] = str(int(multiprocessing.cpu_count()-2))
print("[i]   CPUs available: {}".format(multiprocessing.cpu_count()))
#print("[i]   CPUs used: {}".format(config.get("GENERAL","NTASKS")))
procs = []
for t in range(0,np.shape(ds['th'])[0]):
    ## vis_xzslice(config,ds,ds_lid_list,28)
    proc = multiprocessing.Process(target=vis_xzslice, args=(config,t))
    procs.append(proc)
    proc.start()

# - Complete processes - #
for proc in procs:
    proc.join()
print("[i]   Done!")

[i]   CPUs available: 256
[i]   Done!


In [11]:
dsxy.zcrtopo[0,:,:].min()

<xarray.DataArray 'zcrtopo' ()>
array(nan)

In [7]:
## pip install imageio[ffmpeg]
## import imageio
import imageio.v2 as imageio

output_folder = "./data/xzslice"
filenames    = sorted(os.listdir(output_folder))
fps          = 10

with imageio.get_writer(os.path.join(output_folder,"animation.mp4"), fps=fps) as writer: # duration=1000*1/fps
    for filename in filenames:
        if filename.endswith(".png"):
            image = imageio.imread(os.path.join(output_folder, filename))
            writer.append_data(image)

# imageio.mimsave(image_folder + "/era5_sequence.gif", images, duration=1/fps, palettesize=256/2)  # loop=0, quantizer="nq", palettesize=256
print("MP4 Video created successfully!")

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (939, 960) to (944, 960) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x55307c0] Warning: data is not aligned! This can lead to a speed loss


MP4 Video created successfully!


In [12]:
ds

<xarray.Dataset>
Dimensions:  (t: 101, z: 301, x: 512, y: 512)
Coordinates:
    zcr      (t, z, x) float32 0.0 0.0 0.0 0.0 0.0 ... 120.0 120.0 120.0 120.0
    xcr      (y, x) float32 -1.022e+03 -1.018e+03 ... 1.018e+03 1.022e+03
    ycr      (y, x) float32 -1.022e+03 -1.022e+03 ... 1.022e+03 1.022e+03
Dimensions without coordinates: t, z, x, y
Data variables:
    u        (t, z, x) float32 ...
    v        (t, z, x) float32 ...
    w        (t, z, x) float32 ...
    ue       (t, z, x) float32 ...
    ve       (t, z, x) float32 ...
    the      (t, z, x) float32 ...
    th       (t, z, x) float32 ...
Attributes: (12/45)
    nt:       2000
    nplot:    2000
    nstore:   100000
    nlid:     4
    nslice:   20
    ibcx:     0
    ...       ...
    u0z:      0.0
    v0z:      0.0
    pref00:   100000.0
    nx:       512
    ny:       512
    nz:       301